In [16]:
from wordcloud import WordCloud
from matplotlib import pyplot as plt

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import japanize_matplotlib 
from scipy import optimize
from scipy import stats
plt.style.use("ggplot")
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12
#plt.rcParams['font.family'] = 'IPAexGothic'

import warnings
warnings.filterwarnings('ignore') # warningが出ないように設定
pd.set_option("display.max_rows", None) # pandasの表示上限をなくす
pd.set_option("display.max_columns", None) # pandasの表示上限をなくす
import pickle
import gspread
import json
from oauth2client.service_account import ServiceAccountCredentials
import collections
import gensim
import random

model = gensim.models.Word2Vec.load("../latest-ja-word2vec-gensim-model/word2vec.gensim.model")

In [17]:
# (1) Google Spread Sheetsにアクセス
def connect_gspread(jsonf,key):
    scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
    credentials = ServiceAccountCredentials.from_json_keyfile_name(jsonf, scope)
    gc = gspread.authorize(credentials)
    SPREADSHEET_KEY = key
    worksheet = gc.open_by_key(SPREADSHEET_KEY).sheet1
    return worksheet

In [18]:
def result_vec_list_generator(imput_word_list):
    result_vec_list  = []

    for w in imput_word_list:

        try:
            w_to_v = model.wv[w]

            for mean_vec in mean_vec_list:
                result_vec_plus = w_to_v - mean_vec 
                result_vec_minus =   w_to_v + mean_vec 
    
                result_vec_list.append(result_vec_plus)
                result_vec_list.append(result_vec_minus)
        except:
            print("Error", w)

    return result_vec_list

In [19]:
# 関係ベクトルと類似度が高い単語をn個上位表示
def output_word_list_generator(result_vec_list):
    output_word_list = []
    max_n = 5

    for vec in result_vec_list:
        most_similar = np.array(list(model.wv.most_similar([vec], [], max_n)))
        #output_word_list.extend(most_similar[:,0].tolist())
        
        a = most_similar[:,1]
        for cos in a:
            if float(cos) >= 0.85: #コサイン類似度が0.8以下を除去
                output_word_list.extend(most_similar[:,0].tolist())
                
    return output_word_list

In [20]:
def ggg(list_taple):
    l = []
    for taple in list_taple:
        a = taple[0]
        l.append(a)
    return l

In [21]:
# ここでjsonfile名と2-2で用意したkeyを入力
jsonf = "quickstart-1595337771922-22c2ceb47d3c.json"
spread_sheet_key = "1WKpQ4sxL_IP9Fm6551xDRfhxhDLzmgGAG3FCzDFmamI"
worksheet = connect_gspread(jsonf,spread_sheet_key)

In [22]:
# スプレッドシートのデータを取得するコード
# {"answer":[imput_word_list], ...}

data_dic = {}

for i in range(2, 32): #すプレッソシートの行数
    answers_list = worksheet.row_values(i)
    imput_word_list = answers_list[2:]
    answer_word = answers_list[1]
    
    data_dic[answer_word] = imput_word_list

#  クラスタの評価

In [23]:
f = open("./sample_relation_vec_list.binaryfile",'rb')
#f = open("../kansei_relation_vec_list.binaryfile",'rb')
mean_vec_list = pickle.load(f)

#mean_vec_list = random.sample(mean_vec_list, int(len(mean_vec_list)/30))
len(mean_vec_list)

90

In [24]:
count = 0

for answer_word, imput_word_list in data_dic.items():
    print(answer_word)
    print(imput_word_list)
    
    result_vec_list = result_vec_list_generator(imput_word_list)
    output_word_list = output_word_list_generator(result_vec_list)
    
    c = collections.Counter(output_word_list)
    dic = dict(c)
    dic2 = sorted(dic.items(), key=lambda x:x[1], reverse=True)
    l = ggg(dic2[:30])
    
    for imput_word in imput_word_list:
        l = [output_word for output_word in l if output_word != imput_word]
    
    print(l)
    
    for w in l:
      if answer_word in w: 
        print("成功**************************")
        count += 1
    
    print("　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　")
    
print("正答率", count/len(data_dic))

LIXIL
['トステム', 'INAX', '新日軽', 'YKK']
['LIXIL', '積水ハウス', '大丸松坂屋百貨店', '住友商事', 'Darrell', '第一生命保険', '明治安田生命保険', '三井住友銀行', '（株）', '2451', 'Maggiore', 'クピド', 'コマルカ・デ・サンティアーゴ', '5204', '吉家', '明正', '山之口町', 'きんらん', 'YMFG', '常陽銀行', '親和銀行', 'ブルーステージ', '7550', '率川', '日立化成', '東京海上']
成功**************************
　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　
ムスリム
['イスラム教', 'イスラム', 'アラビア', 'アジア']
['イスラーム', 'ムスリム', 'ヒンドゥー', '仏教徒', 'イスラム教徒', 'スカンディナヴィア', 'フェニキア', 'ペルシャ', 'アジア地域', '東南アジア', '東アジア', 'スカンジナビア', '諸民族', 'アラブ', 'ラテンアメリカ', 'ユダヤ教', 'ワッハーブ派', '西アジア', '中南米', 'アラブ人', 'アラブ世界', '異教徒', 'スンナ派', 'ケルト', '多神教', '宗教']
成功**************************
　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　
ダイドードリンコ
['キリンビバレッジ', 'ダイドーブレンド', 'サントリー', '日本たばこ産業', 'JT', 'コカ・コーラ']
Error ダイドーブレンド
['アサヒビール', 'P&G', '日清食品', '日本コカ・コーラ', 'サッポロビール', 'コカコーラ', '住友商事', 'ペプシ', '雪印乳業', 'アサヒ飲料', '麒麟麦酒', '三洋電機', 'カップヌードル', 'バーガーキング', '清涼飲料水', 'キユーピー', '花王', 'ハウス食品', '資生堂', '秋田銀行', '三菱商事', '京セラ', 'カルチュア・コンビニエンス・

['江戸川乱歩賞', '芥川龍之介賞', '三島由紀夫賞', '新潮社', '中央公論社', '週刊朝日', '文藝賞', '光文社', '裒藁', '己丑', '新潮新人賞', '山本周五郎賞', 'サンデー毎日', '朝日新聞出版', '読売文学賞', '佳作', '二十八', '勝忠', '宇久氏', '曳馬', '覚禅', '幻冬舎', '双葉社', '卅', 'バトルギア', '檜前']
　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　
グーグル
['AI', 'オセロ']
['人工知能', 'メタルギア', '麻雀', 'ロボット', 'AS', 'マジシャン', 'スポーツ漫画', 'ハイパー', 'アサルト', 'サイバー', 'シャドウ', 'アンドロイド', 'コーディネイター', 'MS', 'SMP', 'バイオニック', 'ターミネーター', 'GNドライヴ', 'ELS', 'PSI', 'After Effects', 'ブースター', 'スコープ', 'イオンエンジン', 'キャリアー', 'F-X', 'ゲームエンジン', 'ファイター']
　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　
ソフトバンク
['ヤフー', '孫正義', 'ジェイフォン', 'ボーダフォン']
['イー・モバイル', 'ソフトバンクモバイル', '北長野駅', 'スタンド・アップ', 'Kazimierz', 'Lucio', '宇久氏', 'イー・アクセス', 'DDIポケット', 'ウィルコム', '堀江貴文', '1353', 'Discreet', 'ワイモバイル', '7550', '8450', 'ゴールデンナイター', 'Franks', 'ゾンダ', 'ジーン・アーサー', '桜井古墳', 'ラダマンテュス', 'YOSHI-HASHI', 'トマス・オブ・ウッドストック', 'ファイナルファンタジーVI', 'GDDR5']
成功**************************
　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　

# outlaierの評価

In [27]:
f = open("./sample_relation_outlaier_vec_list.binaryfile",'rb')
#f = open("../kansei_relation_vec_list.binaryfile",'rb')
mean_vec_list = pickle.load(f)

mean_vec_list = random.sample(mean_vec_list, int(len(mean_vec_list)/10))
len(mean_vec_list)

595

In [ ]:
count = 0

for answer_word, imput_word_list in data_dic.items():
    print(answer_word)
    print(imput_word_list)
    
    result_vec_list = result_vec_list_generator(imput_word_list)
    output_word_list = output_word_list_generator(result_vec_list)
    
    c = collections.Counter(output_word_list)
    dic = dict(c)
    dic2 = sorted(dic.items(), key=lambda x:x[1], reverse=True)
    l = ggg(dic2[:30])
    
    for imput_word in imput_word_list:
        l = [output_word for output_word in l if output_word != imput_word]
    
    print(l)
    
    for w in l:
      if answer_word in w: 
        print("成功**************************")
        count += 1
    
    print("　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　")
    
print("正答率", count/len(data_dic))

LIXIL
['トステム', 'INAX', '新日軽', 'YKK']
['LIXIL', 'マツダ', 'ストアー', 'クレディセゾン', 'ダイハツ', '日産', 'キリンビバレッジ', 'P&G', 'サントリー', '三越伊勢丹', '三菱商事', '伊藤忠商事', '野村證券', '日本コカ・コーラ', 'カルビー', 'セブン&アイ・ホールディングス', '京都銀行', 'カルチュア・コンビニエンス・クラブ', 'シボレー', 'トヨタ', '興銀', '住友銀行', 'ダイキン工業', '太平寺', '株式会社ゆうちょ銀行', '大分銀行', '川崎財閥', '松木屋']
成功**************************
　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　
ムスリム
['イスラム教', 'イスラム', 'アラビア', 'アジア']
['イスラーム', 'ムスリム', 'ヒンドゥー教徒', 'イスラム教徒', 'ラテンアメリカ', 'アルメニア', 'アラブ人', 'ヒンドゥー', 'スカンディナヴィア', '東南アジア', '西ヨーロッパ', 'シリア', 'ギリシャ', 'ウイグル', 'モンゴル', 'モンゴル高原', 'ヨーロッパ', 'クシャーナ朝', 'フェニキア', '南インド', '東アジア', 'アジア地域', '混住', '他民族', '多民族', 'キリスト教徒']
成功**************************
　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　
ダイドードリンコ
['キリンビバレッジ', 'ダイドーブレンド', 'サントリー', '日本たばこ産業', 'JT', 'コカ・コーラ']
Error ダイドーブレンド
['P&G', '日本コカ・コーラ', '日清食品', 'アサヒビール', '京セラ', 'カルビー', '住友商事', 'パナソニック', '三洋', 'NTT', '日産自動車', 'ヤマハ発動機', '三洋電機', '日本IBM', 'コカコーラ', 'ENEOS', '本田技研工業', 'DDI', 'ユニクロ', 'サッポ

# outlaierの評価